In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cftime

In [2]:
from osprey.actions.reader import reader_nemo_field

In [3]:
data = reader_nemo_field(expname='pi00', startyear=1990, endyear=1999, varname='thetao')

2024-12-19 12:35:06,151 - INFO - Data available in the range 1990-1999.


In [4]:
from osprey.means.means import timemean

In [7]:
ave = timemean(data=data, format='winter', use_cftime=True)
ave

<xarray.DataArray 'thetao' (time: 10, z: 31, y: 148, x: 180)> Size: 33MB
dask.array<getitem, shape=(10, 31, 148, 180), dtype=float32, chunksize=(1, 31, 148, 180), chunktype=numpy.ndarray>
Coordinates:
    lat      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
    lon      (y, x) float32 107kB dask.array<chunksize=(148, 180), meta=np.ndarray>
  * z        (z) float32 124B 5.0 15.0 25.0 35.01 ... 4.25e+03 4.75e+03 5.25e+03
  * time     (time) object 80B 1990-01-15 12:00:00 ... 1999-01-15 12:00:00
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           temperature
    units:               degC
    online_operation:    average
    interval_operation:  5400 s
    interval_write:      1 month
    cell_methods:        time: mean (interval: 5400 s)

In [8]:
ave['time']

<xarray.DataArray 'time' (time: 10)> Size: 80B
array([cftime.DatetimeGregorian(1990, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1991, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1992, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1993, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1994, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1995, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1996, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1997, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1998, 1, 15, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1999, 1, 15, 12, 0, 0, 0, has_year_zero=False)],
      dtype=object)
Coordinates:
  * time     (time) object 80B 1990-01-15 12:00:00 ... 1999-01-15 12:00:00